<a href="https://colab.research.google.com/github/dr-mushtaq/Math-QA-Difficulty-Classifier/blob/main/Math_QA_Difficulty_Classifier_(1%E2%80%935)_using_Transformersi_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**🔧 Step-by-Step Implementation Plan**

📁 1. Dataset Preparation

➤ **Data Sources**
Train: hendrycks_math_train.csv

Test: hendrycks_math_test.csv

➤ **Tasks:**

Load both CSVs using pandas

Inspect and clean the data (e.g., missing values)

Combine question + answer as input text

Difficulty is the label (1–5)

**🧠 2. Modeling Approach**

We’ll treat this as a text classification task, using transformer-based models like:

BERT, RoBERTa, or DeBERTa (for accuracy)

DistilBERT (for speed and low compute environments)

Fine-tune using HuggingFace Transformers

In [ ]:
➤ Input:

plaintext
Input: "Q: <question text> A: <answer text>"
Output: Class label (1 to 5)

**🧪 3. Evaluation Metric**

Accuracy

Weighted F1-score (due to potential class imbalance)

Confusion Matrix

**📦 4. Required Libraries**

In [1]:
pip install transformers datasets scikit-learn pandas torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

**✅ Example Python Code (Core)**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Load data
df = pd.read_csv("https://raw.githubusercontent.com/dr-mushtaq/Math-QA-Difficulty-Classifier/main/Dataset/hendrycks_math_train%20(1).csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/dr-mushtaq/Math-QA-Difficulty-Classifier/main/Dataset/hendrycks_math_test%20(1).csv")

# Combine problem and solution
df["text"] = "Q: " + df["problem"] + " A: " + df["solution"]
df_test["text"] = "Q: " + df_test["problem"] + " A: " + df_test["solution"]

# Prepare train/val split
train_texts, val_texts, train_labels, val_labels = train_test_split(df["text"], df["level"], test_size=0.2, random_state=42)

# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(df_test["text"]), truncation=True, padding=True)

# Dataset wrapper
class MathDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels if labels is not None else None

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx] - 1)  # make 0-indexed
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = MathDataset(train_encodings, train_labels.tolist())
val_dataset = MathDataset(val_encodings, val_labels.tolist())

# Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch", # Added to match eval_strategy
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

# Evaluation
preds = trainer.predict(val_dataset)
y_pred = preds.predictions.argmax(-1) + 1  # return to 1-indexed
y_true = val_labels.values

print(classification_report(y_true, y_pred, digits=3))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-5-3711130817.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:


Abort: 